In [1]:
# итоговый код
import numpy as np
import re
import random
from collections import Counter
import codecs
from glob import glob
import pickle
import hashlib 
import os
from pathlib import Path

# Регулярное выражение для извлечения слов из кода
split_re = re.compile(r"([^\w\d\-])")

In [2]:
# Функция создания словаря вероятностей появления символа, нужна для того, 
# чтобы в последующем считать энтропию символов строки
# На вход она получает путь к файлам, а возвращает словарь,
# где символы - это ключи, а вероятность встречи - значение по ключу
def Create_Dictionary(files):
    charprobabilitydict = {}
    files = Path(files)
    for path_object in files.glob('**/*.php'): # Пробегаемся по всем php файлам во всех подпапках заданного пути
        if path_object.is_file():
            with codecs.open(path_object, 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read()
                a = Counter(text)
                for key in a:
                    if (key in charprobabilitydict) != 1:
                        charprobabilitydict[key] = a[key] / len(text) 
                    else:
                        charprobabilitydict[key] = (charprobabilitydict[key] + a[key] / len(text)) / 2
            
    return charprobabilitydict

In [3]:
# Функция нужна для подсчёта вероятностей символов в новых файлах и
# добавления их в уже существующий словарь
# На вход принимает существующий словарь и файл, из которого
# мы считываем вероятности символов
# Возвращает она изменённый словарь
def Dictionary_update(charprobabilitydict, file):
    with codecs.open(file, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
            a = Counter(text)
            for key in a:
                if (key in charprobabilitydict) != 1:
                    charprobabilitydict[key] = a[key] / len(text) 
                else:
                    charprobabilitydict[key] = (charprobabilitydict[key] + a[key] / len(text)) / 2
    
    return charprobabilitydict

In [4]:
# Функция нарезки строк из наших файлов, принимает на вход строку и словарь вероятностей,
# нарезает строку на слова, после этого считает посимвольную энтропию строки
# Возвращает список, где первый элемент в списке - это энтропия строки,
# а все после последующие элементы - слова строки
def spliter(code, charprobablitydict):
    code = str(code).strip(r"\\r\\n").strip(r"\r\n").strip("\n").strip("\\n")
    for i in range(10):
        sum = 0
        for j in range(len(code)):
            sum += - np.sum(float(charprobablitydict[code[j]]) * np.log2(float(charprobablitydict[code[j]]))) # считает энтропию
    return [sum] + [w for w in split_re.split(code) if w and w != " "]

In [5]:
# Функция создания матрицы из строк файла, 
# на вход принимает php файл, словарь вероятностей и кол-во слов в строке(выбирали сами)
# После чего из строк собирает матрицу 
# Выводит матрицу из всех строк файла нарезанные функцией spliter
def Entropy_Matrix(file, charprobabilitydict, number_of_characters):
    with codecs.open(file, 'r', encoding='utf-8', errors='ignore') as f:
        matrix_entropy = []
        for line in f.readlines():
            parsed = spliter(line[0:number_of_characters], charprobabilitydict)
            if parsed and parsed[0] != 0:
                matrix_entropy.append(parsed)
    
    return matrix_entropy

In [1]:
# Функция выбора более важных для нас строк, мы добавили опасные функции, которые расположены в определённом порядке,
# от самых опасных к менее опасным. 
# Сначала мы добавляем в файл все строки с опасными функциями(но не больше 20), после этого добавляем строки
# со средним значением энтропии(кол-во строк регулирует показатель len_str), после этого добавляем такое же кол-во строк
# с минимальным и максимальным значением энтропии, после этого дозаполняем матрицу строк до 40 штук рандомными значениями
# На вход функция получает матрицу из всех строк файла и кол-во строк каждой категории, возвращает новую матрицу из 40 строк
def Slicing(entropy_matrix, len_str):
    new_matrix = []
    entropy_matrix = np.sort(entropy_matrix, axis=0)
    average_value = 0
    # Список опасных функций
    danger_functions = ['pcntl_exec', 'proc_open', 'passthru', 'shell_exec', 'popen', 'exec', 'system', 'create_function', \
                        'assert', 'preg_replace', 'eval', 'include', 'include_once', 'require', 'require_once', 'array_walk', \
                        'array_map', 'phpinfo', 'mail', 'header', 'fopen', 'tmpfile', 'copy', 'extract', 'file_put_contents', \
                        'move_uploaded_file', 'symlink', 'touch', 'file_get_contents', 'chmod', 'ob_start', 'assert_options', \
                        'register_shutdown_function', 'register_tick_function', 'uksort', 'usort', 'preg_replace_callback', \
                        'iterator_apply', 'call_user_func', 'call_user_func_array', 'get_current_user']
    index = []
#     Если файл менее 40 строк, то добавляем все
    if len(entropy_matrix) <= 40:
        new_matrix = entropy_matrix
    else:
#     Сначала добавляем строки содержащие опасные функции    
        for i in range(len(entropy_matrix)):
            if len(index) > 20:
                break
            for func in danger_functions:
                if func in entropy_matrix[i]:
                    new_matrix.append(entropy_matrix[i])
                    index.append(i)
                break
        for i in index:
            entropy_matrix = np.delete(entropy_matrix, i)

#     Теперь добавляем строки со средним значением
        for i in range(len(entropy_matrix)):
            average_value += entropy_matrix[i][0]
        average_value = average_value / len(entropy_matrix)
      
        for i in range(len_str):
            for i in range(len(entropy_matrix) - 1):
                if (entropy_matrix[i][0] <= average_value) and (entropy_matrix[i + 1][0] >= average_value):
                    new_matrix.append(entropy_matrix[i])
                    entropy_matrix = np.delete(entropy_matrix, i)
                    
#   Добавляем строки с максимальным и минимальным значением
        for i in range(len_str):
            new_matrix.append(entropy_matrix[i])
            entropy_matrix = np.delete(entropy_matrix, i)
            new_matrix.append(entropy_matrix[len(entropy_matrix) - i - 1])
            entropy_matrix = np.delete(entropy_matrix, len(entropy_matrix) - i - 1)
#   Оставшиеся строки докидываем рандомно(до 40 во сём файле)
        for i in range(25 - len(index)):
            k = random.randint(0,len(entropy_matrix) - 1)
            new_matrix.append(entropy_matrix[k])
            entropy_matrix = np.delete(entropy_matrix, k)

    return new_matrix

In [7]:
# Эта функция добавляет к нашей матрице файла ещё 1 строку, которая содержит в первой ячейке - категорию файла,
# во-второй ячейке - энтропию пути и названия файла, а в третьей - путь и название файла, разбитые на слова
# На вход получает нашу отредактированную матрицу из 40 строк файла, сам Path файла и словарь вероятностей(для подсчёта энтропии)
# На выходе выдаёт матрицу из 41 строки файла
def Path_matrix(new_matrix, file, dictionary):
    path_matrix = []
    file_stats = os.stat(file) # Посчитали вес файла
    first = file_stats.st_size / 1000
    # Распределяем файл по категориям по размеру на диске, всего есть 41 категория
    # распределение происходит так, если файл весит меньше 1 Кб - то это 0 категория, если меньше 2Кб - то это 2 и т.д.
    category = 0
    if first <= 1:
        category = 0
    elif first <= 10:
        category = round(first)
    elif first <= 100:
        category = round(first / 10) + 10
    elif first <= 1000:
        category = round(first / 100) + 20
    elif first <= 10000:
        category = round(first / 1000) + 30
    else:
        category = 41
# Добавляем строку с категорией файла в первой ячейке, а дальше нарезаем её, как делали раньше с помощью функции spliter
    path_matrix.append([category] + spliter(str(file).split("\\", 5)[-1], dictionary))
    for i in new_matrix:
        path_matrix.append(i)
    return path_matrix

In [8]:
# Функция нужна для того, чтобы привязать нашу матрицу файла к самому файлу, делается это с помощью словаря,
# где ключ - это hash файла, а значение по ключу - матрица файла
# На вход функция получает сам файл, матрицу файла и словарь, куда будет записан сам файл с его матрицей
# Выводит функция словарь dict[hash] = matrix
def Dict_hash(file, entropy_matrix, dict_hash):
    with open(file,"rb") as f:
        bytes = f.read() 
        readable_hash = hashlib.sha256(bytes).hexdigest();
        dict_hash[readable_hash] = entropy_matrix
    
    return dict_hash


In [2]:
# Функция записывает словарь с файлами в отдельный файл
# На вход получает сам словарь и файл, в который она его запишет
def Import_file(dict_hash, file):
    
    with open(file, 'wb') as f:
        pickle.dump(dict_hash, f)
    
    return

In [3]:
# Функция считывает содержимое файла, на вход получает файл, откуда нужно считать информацию,
# а выводит всё содержимое файла в переменную
def Export_file(dictionary_file):
    
    with open(dictionary_file,'rb') as f:
        der = pickle.load(f)

    return der 

# Tests

In [11]:
import unittest

# Unitets

class TestNotebook(unittest.TestCase):
    
    def test_Create_Dictionary(self):
        self.assertIsNotNone(Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux\safe'))
        self.assertDictEqual(Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux\safe'), Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux\safe'))
        self.assertTrue(type(Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux\safe')) is dict)
        self.assertTrue(type(Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux\safe')['!']) is float)
    
    
    def test_Dictionary_update(self):
        dictionary = Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux\safe')
        new_dictionary = Dictionary_update(dictionary, r'C:\Users\rtyde\Cloud\CloudLinux\wordpress-5.8\wordpress\index.php')
        
        self.assertIsNotNone(new_dictionary)
        self.assertFalse(self.assertDictEqual(dictionary, new_dictionary))
        self.assertTrue(type(new_dictionary) is dict)
        self.assertNotEqual(dictionary[';'], Dictionary_update(dictionary, r'C:\Users\rtyde\Cloud\CloudLinux\wordpress-5.8\wordpress\wp-load.php')[';'])


    def test_spliter(self):
        dictionary = Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux')
        split = spliter(r'C:\Users\rtyde\Cloud\CloudLinux\wordpress-5.8\wordpress\wp-load.php', dictionary)
    
        self.assertIsNotNone(split)
        self.assertTrue(type(split) is list)
        self.assertTrue(type(split[0]) is np.float64)
    
    
    def test_Entropy_Matrix(self):
        dictionary = Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux')
        entropy_matrix = Entropy_Matrix(r'C:\Users\rtyde\Cloud\CloudLinux\wordpress-5.8\wordpress\wp-load.php', dictionary, 250)
        
        self.assertIsNotNone(entropy_matrix)
        for i in entropy_matrix:
            self.assertTrue(type(i[0]) is np.float64)
            self.assertTrue(type(i[1]) is str)
        self.assertTrue(type(entropy_matrix) is list)
        
    
    def test_Slicing(self):
        file= r'C:\Users\rtyde\Cloud\CloudLinux\wordpress-5.8\wordpress\wp-load.php'
        dictionary = Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux')
        entropy_matrix = Entropy_Matrix(file, dictionary, 250)
        slic = Slicing(entropy_matrix, 5)
        f = open(file, 'r')
        len_count = sum(1 for line in f)
        
        self.assertIsNotNone(slic)
        if  len_count >= 40:
            self.assertEqual(len(slic), 40)
        else:
            self.assertEqual(len(slic), len_count)
        f.close()
        self.assertTrue(type(slic) is list)
        
        
    def test_Path_matrix(self):
        file= r'C:\Users\rtyde\Cloud\CloudLinux\wordpress-5.8\wordpress\wp-load.php'
        dictionary = Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux')
        entropy_matrix = Entropy_Matrix(file, dictionary, 250)
        slic = Slicing(entropy_matrix, 5)
        path_matrix = Path_matrix(slic, file, dictionary)
        
        self.assertIsNotNone(path_matrix)
        self.assertTrue(type(path_matrix[0][0]) is int)
        self.assertTrue(type(path_matrix[0][1]) is np.float64)
        self.assertTrue(type(path_matrix[0][2]) is str)
        self.assertEqual(len(slic) + 1, len(path_matrix))
        
        
    def test_Dict_hash(self):
        file= r'C:\Users\rtyde\Cloud\CloudLinux\wordpress-5.8\wordpress\wp-load.php'
        dictionary = Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux')
        entropy_matrix = Entropy_Matrix(file, dictionary, 250)
        slic = Slicing(entropy_matrix, 5)
        path_matrix = Path_matrix(slic, file, dictionary)
        dict_hash = {}
        res_dict = Dict_hash(file, path_matrix, dict_hash)
        
        self.assertIsNotNone(res_dict)
        self.assertTrue(type(res_dict) is dict)
        self.assertTrue(type(res_dict[list(res_dict.keys())[0]]) is list)
        self.assertEqual(len(res_dict[list(res_dict.keys())[0]]), len(path_matrix))
        
        
    def test_Import_file(self):
        file= r'C:\Users\rtyde\Cloud\CloudLinux\wordpress-5.8\wordpress\wp-load.php'
        dictionary = Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux')
        entropy_matrix = Entropy_Matrix(file, dictionary, 250)
        slic = Slicing(entropy_matrix, 5)
        path_matrix = Path_matrix(slic, file, dictionary)
        dict_hash = {}
        res_dict = Dict_hash(file, path_matrix, dict_hash)
        dictionary_file = Import_file(res_dict, 'dict_hash.txt')
        
        with open('dict_hash.txt','rb') as f:
            der = pickle.load(f)
            self.assertIsNotNone(der)
            self.assertDictEqual(der, res_dict)
            self.assertTrue(type(der[list(der.keys())[0]]) is list)
            
            
    def test_Export_file(self):
        file= r'C:\Users\rtyde\Cloud\CloudLinux\wordpress-5.8\wordpress\wp-load.php'
        dictionary = Create_Dictionary(r'C:\Users\rtyde\Cloud\CloudLinux')
        entropy_matrix = Entropy_Matrix(file, dictionary, 250)
        slic = Slicing(entropy_matrix, 5)
        path_matrix = Path_matrix(slic, file, dictionary)
        dict_hash = {}
        res_dict = Dict_hash(file, path_matrix, dict_hash)
        dictionary_file = Import_file(res_dict, 'dict_hash.txt')
        res = Export_file('dict_hash.txt')
        
        self.assertIsNotNone(res)
        self.assertTrue(type(res) is dict)
        self.assertDictEqual(res, res_dict)
    
unittest.main(argv=[''], verbosity=2, exit=False)

test_Create_Dictionary (__main__.TestNotebook) ... ok
test_Dict_hash (__main__.TestNotebook) ... C:\Users\rtyde\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
ok
test_Dictionary_update (__main__.TestNotebook) ... ok
test_Entropy_Matrix (__main__.TestNotebook) ... ok
test_Export_file (__main__.TestNotebook) ... ok
test_Import_file (__main__.TestNotebook) ... ok
test_Path_matrix (__main__.TestNotebook) ... ok
test_Slicing (__main__.TestNotebook) ... ok
test_spliter (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 9 tests in 7.461s

OK


# Creating dictionares

In [ ]:
# Создание словаря вероятностей 
files = Path(r'C:\Users\rtyde\Cloud\CloudLinux')
dictionary = Create_Dictionary(files)

In [ ]:
# Добавление в словарь вероятностей файлов разных типов(SA, SAFE,INJ,UNK)
files = Path(r'C:\Users\rtyde\Cloud\CloudLinux\Test')
for path_object in files.glob('**/'+h):
    if path_object.is_file():
        dictionary = Dictionary_update(dictionary, path_object)
Import_file(dictionary, 'dictionary.txt') 

In [ ]:
# Импортируем словарь вероятностей в файл
dictionary = Export_file('dictionary.txt')

In [ ]:
# Создание словаря SA файлов(файлы вордпресса) и импорт словаря в файл
files = Path(r'C:\Users\rtyde\Cloud\CloudLinux\Test\SA_phpfiles')
h = '?'*64 # Это маска, потому что не у всех файлов есть расширение
dict_hash = {}
for path_object in files.glob('**/'+h):
    if path_object.is_file():
        print(path_object)
        matr = Path_matrix(Slicing(Entropy_Matrix(path_object, dictionary, 250), 5), path_object, dictionary)
        dict_hash = Dict_hash(path_object, matr, dict_hash)
Import_file(dict_hash, 'SA_Test.txt')        
print(dict_hash)

In [ ]:
# Создание словаря INJ файлов и импорт словаря в файл
files = Path(r'C:\Users\rtyde\Cloud\CloudLinux\Test\INJ_phpfiles')
h = '?'*64
dict_hash = {}
for path_object in files.glob('**/'+h):
    if path_object.is_file():
        print(path_object)
        matr = Path_matrix(Slicing(Entropy_Matrix(path_object, dictionary, 250), 5), path_object, dictionary)
        dict_hash = Dict_hash(path_object, matr, dict_hash)
Import_file(dict_hash, 'INJ_Test.txt') 
print(dict_hash)

In [ ]:
# Создание словаря SAFE файлов(это файлы и гугл диска) и запись словаря в файл
files = Path(r'C:\Users\rtyde\Cloud\CloudLinux\Test\SAFE_phpfiles')
h = '?'*64
dict_hash = {}
for path_object in files.glob('**/'+h):
    if path_object.is_file():
        print(path_object)
        matr = Path_matrix(Slicing(Entropy_Matrix(path_object, dictionary, 250), 5), path_object, dictionary)
        dict_hash = Dict_hash(path_object, matr, dict_hash)
Import_file(dict_hash, 'SAFE_Test.txt') 
print(dict_hash)

In [ ]:
# Создание словаря UNK файлов и импорт его в файл
files = Path(r'C:\Users\rtyde\Cloud\CloudLinux\Test\UNK_phpfiles')
h = '?'*64
dict_hash = {}
for path_object in files.glob('**/'+h):
    if path_object.is_file():
        print(path_object)
        matr = Path_matrix(Slicing(Entropy_Matrix(path_object, dictionary, 250), 5), path_object, dictionary)
        dict_hash = Dict_hash(path_object, matr, dict_hash)
Import_file(dict_hash, 'UNK_Test.txt') 
print(dict_hash)

In [ ]:
# Создание словаря SAFE файлов WordPress-a и запись его в файл
files = Path(r'C:\Users\rtyde\Cloud\CloudLinux')
dict_hash = {}
for path_object in files.glob('**/*.php'):
    if path_object.is_file():
        print(path_object)
        matr = Path_matrix(Slicing(Entropy_Matrix(path_object, dictionary, 250), 5), path_object, dictionary)
        dict_hash = Dict_hash(path_object, matr, dict_hash)
Import_file(dict_hash, 'WordPress.txt')        
print(dict_hash)

# TF-IDF functions 

In [4]:
import numpy as np
import pickle
from collections import Counter
import math
import pandas as pd

In [5]:
# Выгружаем словари
dict_hash = Export_file('dict_hash(old).txt')
SA = Export_file('SA_Test.txt')
SAFE = Export_file('SAFE_Test.txt')
UNK = Export_file('UNK_Test.txt')
INJ = Export_file('INJ_Test.txt')

In [ ]:
# Функция для статистического взвешивания элементов матрицы с помощью TF-IDF для словарей файлов, она создаёт новый словарь, где на вход подаётся наш словарь файлов.
# На выходе функция создаёт новый словарь, ключ нового словаря - hash файла,
# а значение - это матрица из словарей, где по ключу мы вызываем значение из матрицы нашего прошлого словаря.
# Вывод примерно такой:
# print(dict_Tf_IDF['00731ed48b28b30f4583466f631489519325c77c0bc68597c2e00add00236248'])
#
#[{1: 0.1858614279519973, 4.123289312938014: 0.1858614279519973, 'safe': 0.1858614279519973, '\\': 0.11770124870633125,
#  'safe_0395775733f263f35c5d5d0fe1c9c2748066ff09b556558d530f5657b9cb07f4': 0.1858614279519973, '.': 0.05684857266743394,
#  'php': 0.11770124870633125}, {0.022447084697324208: 0.6505149978319906, '}': 0.5}, {0.09590458196907431: 0.4336766652213271,
#  '<': 0.20068666377598746, '?': 0.27463624698143957}, {0.1141567729236648: 0.4336766652213271, '?': 0.27463624698143957,
#  '>': 0.20068666377598746}, {1.5678506668856307: 0.6505149978319906, '}': 0.5}, {2.486148620051607: 0.10841916630533177,
#  'if': 0.10841916630533177, '(': 0.03799432963747703, '$': 0.01848739580136303, '_POST': 0.043573228773361464,
#  '[': 0.043573228773361464, "'": 0.06632333477867293, 'name': 0.050171665943996864, ']': 0.043573228773361464,
#  ')': 0.03799432963747703, '{': 0.08333333333333333}]

def TF_IDF(dict_hash):
    res_dict = {}

    def compute_tf(text): # Подсчёт TF
        tf_text = Counter(text)
        for i in tf_text:
            tf_text[i] = tf_text[i]/float(len(text))
        return tf_text
    
    def compute_idf(word, dict_hash): # Подсчёт IDF
        return math.log10(len(dict_hash)/sum([1.0 for i in dict_hash if word in i]))
    

    for i in dict_hash.keys():
        documents_list = []
        for text in dict_hash[i]:
            tf_idf_dictionary = {}
            computed_tf = compute_tf(text)
            for word in computed_tf:
                tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, dict_hash[i])
            documents_list.append(tf_idf_dictionary)
        res_dict[i] = documents_list
            
    return res_dict

Function for creating result dictionary with TF-IDF tokens

In [ ]:
# Функция преобразует из двух словарей: исходного и полученного с TF-IDF векторизацией в один словарь
# Это нужно, чтобы сохранить значения энтропии и категории файла, и не заменить их на их значения TF-IDF
# На вход подаются два словаря с файлами - исходный и словарь с TF-IDF
# Функция возвращает итоговый словарь
def Merge_dict(dict_hash, new_dict): 
    res_dict = {}

    for i in dict_hash.keys():
        matr = []
        for j in range(len(dict_hash[i])):
            matrix = []
            for k in range(len(dict_hash[i][j])):
                if j == 0:
                    if k == 0 or k == 1:
                        matrix.append(dict_hash[i][j][k])
                    else:
                        matrix.append(new_dict[i][j][dict_hash[i][j][k]])
                else:
                    if k == 0:
                        matrix.append(dict_hash[i][j][k])
                    else:
                        matrix.append(new_dict[i][j][dict_hash[i][j][k]])
            matr.append(matrix) 
            res_dict[i] = matr
    return res_dict

In [ ]:
# Создаём словари TF-IDF
new_dict = TF_IDF(dict_hash)
SA_dict = TF_IDF(SA)
SAFE_dict = TF_IDF(SAFE)
UNK_dict = TF_IDF(UNK)
INJ_dict = TF_IDF(INJ)

In [ ]:
# Создаём итоговые словари
res_dict = Merge_dict(dict_hash, new_dict)
res_SA = Merge_dict(SA, SA_dict)
res_SAFE = Merge_dict(SAFE, SAFE_dict)
res_UNK = Merge_dict(UNK, UNK_dict)
res_INJ = Merge_dict(INJ, INJ_dict)

In [ ]:
# Записываем словари в файлы
Import_file(res_dict, 'result_dictionary.txt')
Import_file(res_SA, 'SA_dict.txt')
Import_file(res_SAFE, 'SAFE_dict.txt')
Import_file(res_UNK, 'UNK_dict.txt')
Import_file(res_INJ, 'INJ_dict.txt')

# Creating Dataset

In [ ]:
# Функция создаёт датасет, где hash это лейбл, а матрицу файла разворачиваем и создаём одну длинную строку 41*255
# На вход функция получает файл со словарём, а возвращает DataFrame
def Create_Dataset(file):
    dict_hash = Export_file(file)
    df = pd.DataFrame(columns=np.arange(255*41)) 
    for i in dict_hash.keys():
        res = []
        for line in dict_hash[i]:
            string = np.ones(255) # создаём строку из 255 единиц
            for j in range(len(line)):
                string[j] = line[j] # перезаписываем строку нашего файла туда
            res.extend(string) # добавляем каждую строку в конец следующей
        s = pd.Series(res, name = i)
        df = df.append(s)
    return df

In [ ]:
# Создаём датасеты для каждой категории файлов
df_res = Create_Dataset('result_dictionary.txt')
df_sa = Create_Dataset('SA_dict.txt')
df_safe = Create_Dataset('SAFE_dict.txt')
df_inj = Create_Dataset('INJ_dict.txt')
df_ink = Create_Dataset('UNK_dict.txt')

In [ ]:
# Переименовываем столбец
df_res.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)
df_sa.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)
df_safe.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)
df_inj.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)
df_unk.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)

In [ ]:
#  Создаём столбец категории файла и размечаем файлы
df_res['Category'] = [1 for i in range(len(data_word))]
df_sa['Category'] = [2 for i in range(len(data_SA))]
df_safe['Category'] = [1 for i in range(len(data_SAFE))]
df_inj['Category'] = [3 for i in range(len(data_INJ))]
df_unk['Category'] = [4 for i in range(len(data_UNK))]

In [ ]:
# Объединяем все датасеты в один
data = pd.concat([df_res, df_sa, df_safe, df_inj, df_unk])

In [ ]:
# Заполняем пропущенные значения 
data = data.fillna(1)

In [14]:
# Функция округления списка
def rounding(vect):
    res = []
    for i in vect:
        res.append(round(i))
    return res

In [ ]:
# Функция поиска среднего значения в строке
# На вход подаётся список, а выход - это среднее значение в списке
def Mean_list(dict_hash):
    mean = []
    for line in dict_hash:
        if len(line) > 1:
            mean.append(np.mean(line[1:]))
        else:
            mean.append(np.mean(line))
    return np.mean(mean)

# Функция создания датасета с заполнением пропущенных значений числами из нормального распределения
# На вход подаётся файл со словарём, возвращает функция датафрейм
def Data(file):
    dict_hash = Export_file(file)
    df = pd.DataFrame(columns=np.arange(255*41))
    for i in dict_hash.keys():
        res = []
        for line in dict_hash[i]:
            # Строка заполняется числами из нормального распределения, со средним по строке числом, равным среднему числу
            # в строке словаря файла и ср-кв отклонением - квадрат среднего значения
            string = abs(np.random.normal(scale = Mean_list(dict_hash[i]), loc = Mean_list(dict_hash[i])**2, size = 255))
            for j in range(len(line)):
                string[j] = line[j] # заполняем числами из словаря
            res.extend(string)
        if len(dict_hash[i]) < 41: # проверка на кол-во строк, чтобы добавить пустые строки
            res.extend(abs(np.random.normal(scale = np.mean(res), loc = np.mean(res)**2, size = (41 - len(dict_hash[i])) * 255 )))
        s = pd.Series(res, name = i)
        df = df.append(s)
    return df

In [ ]:
# Создание датафреймов
df_res = Data('result_dictionary.txt')
df_res = Data('result_dictionary.txt')
df_sa = Data('SA_dict.txt')
df_safe = Data('SAFE_dict.txt')

In [ ]:
# Переименовываем столбец
df_res.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)
df_sa.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)
df_safe.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)
# Дописываем категории
df_res['Category'] = [True for i in range(len(df_res))]
df_sa['Category'] = [False for i in range(len(df_sa))]
df_safe['Category'] = [True for i in range(len(df_safe))]
# Объединяем датафреймы
data = pd.concat([df_res, df_sa, df_safe])
# Записываем датасет в файл
Import_file(data, 'Data_Norm.csv')
# Импортируем датасет
data = Export_file('Data_Norm.csv')

In [ ]:
# Функция аналогичная функции для заполнения нормальным распределением, но заполняет пропущенные значения числами из
# Пуассоновского потока событий
def Data_Puasson(file):
    dict_hash = Export_file(file)
    df = pd.DataFrame(columns=np.arange(255*41))
    for i in dict_hash.keys():
        res = []
        for line in dict_hash[i]:
            string = np.random.sample(255)
            for j in range(len(line)):
                string[j] = line[j]
            res.extend(string)
        if len(dict_hash[i]) < 41:
            res.extend(np.random.sample((41 - len(dict_hash[i])) * 255))
        s = pd.Series(res, name = i)
        df = df.append(s)
    return df

In [ ]:
# Действия аналогичные предыдущему заполнению

In [ ]:
df_res = Data_Puasson('result_dictionary.txt')
df_sa = Data_Puasson('SA_dict.txt')
df_safe = Data_Puasson('SAFE_dict.txt')

In [ ]:
df_res.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)
df_sa.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)
df_safe.rename(columns={'Unnamed: 0': 'Hash'}, inplace=True)

In [ ]:
df_res['Category'] = [True for i in range(len(df_res))]
df_sa['Category'] = [False for i in range(len(df_sa))]
df_safe['Category'] = [True for i in range(len(df_safe))]

In [ ]:
data = pd.concat([df_res, df_sa, df_safe])

In [ ]:
Import_file(data, 'Data_Puasson.csv')

In [6]:
data = Export_file('Data_Puasson.csv')

# Модели

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, mean_squared_error, r2_score, classification_report

In [8]:
# Разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data[data.columns[1:-1]], data['Category'], test_size=0.1, random_state=69)

# Наивный байес

In [9]:
model = MultinomialNB().fit(X_train, y_train)
predict = model.predict(X_test)
core = f1_score(y_test, predict, average='weighted')
score = accuracy_score(y_test, predict)
print(core)
print(score)
print(precision_score(predict, y_test, average='weighted'))
print(recall_score(predict, y_test, average='weighted'))

0.6268099282339787
0.6075949367088608
0.5936849649946884
0.6075949367088608


In [10]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

       False       0.35      0.53      0.43        86
        True       0.78      0.63      0.70       230

    accuracy                           0.61       316
   macro avg       0.57      0.58      0.56       316
weighted avg       0.67      0.61      0.63       316



# Логистическая регрессия

In [11]:
model3 = LogisticRegression().fit(X_train, y_train)
predict3 = model3.predict(X_test)
print(model3.score(X_test, y_test))
print(classification_report(y_test, predict3))

0.7531645569620253
              precision    recall  f1-score   support

       False       0.55      0.51      0.53        86
        True       0.82      0.84      0.83       230

    accuracy                           0.75       316
   macro avg       0.69      0.68      0.68       316
weighted avg       0.75      0.75      0.75       316



C:\Users\rtyde\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Случайные леса

In [12]:
model1 =  RandomForestRegressor()
RandomFor = model1.fit(X_train, y_train)
predict1 = model1.predict(X_test)

In [15]:
predict_1 = rounding(predict1)

In [16]:
core1 = f1_score(y_test, predict_1, average='weighted')
score1 = accuracy_score(y_test, predict_1)
print(core1)
print(score1)
print(mean_squared_error(predict_1, y_test))

0.8713728522782372
0.870253164556962
0.12974683544303797


In [17]:
print(classification_report(y_test, predict_1))

              precision    recall  f1-score   support

       False       0.75      0.79      0.77        86
        True       0.92      0.90      0.91       230

    accuracy                           0.87       316
   macro avg       0.83      0.85      0.84       316
weighted avg       0.87      0.87      0.87       316



# Лассо

In [18]:
from sklearn.linear_model import LassoCV
model = LassoCV(normalize = True, tol = 0.01)
model.fit(X_train, y_train)

LassoCV(normalize=True, tol=0.01)

In [19]:
prediction  = model.predict(X_test)
predict_L = rounding(prediction)
print(f1_score(y_test, predict_L, average='weighted'))
print(accuracy_score(y_test, predict_L))
print(mean_squared_error(predict_L, y_test))
print(r2_score(predict_L, y_test))
print(np.mean(predict_L == y_test))

0.7765048413072123
0.7753164556962026
0.22468354430379747
-0.11052813938523975
0.7753164556962026


In [20]:
print(classification_report(y_test, predict_L))

              precision    recall  f1-score   support

       False       0.58      0.60      0.59        86
        True       0.85      0.84      0.84       230

    accuracy                           0.78       316
   macro avg       0.72      0.72      0.72       316
weighted avg       0.78      0.78      0.78       316

